#Collecting Data

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl

In [ ]:
train_dataset = pkl.load(open('/content/drive/MyDrive/mediaeval2016/Twitter/myTemp/train_dataset.pkl', 'rb'))
test_dataset=pkl.load(open('/content/drive/MyDrive/mediaeval2016/Twitter/myTemp/test_dataset.pkl','rb'))

In [ ]:
train_dataset[0]#0:fake,1:Real

['need feds solve #bostonbombing #4chan',
 'need feds solve #bostonbombing #4chan',
 'boston_fake_03.jpg',
 0]

In [ ]:
def get_data(dataset):
  text=[]
  target=[]
  image_name=[]
  for data in dataset:
    text.append(data[1])
    target.append(data[3])
    image_name.append(data[2])
  return text,image_name,target
train_text,train_image_name,train_target=get_data(train_dataset)
test_text,test_image_name,test_target=get_data(test_dataset)
len(train_text),len(test_text)

(10740, 932)

In [ ]:
train_text[:5],train_image_name[:5],train_target[:5]

(['need feds solve #bostonbombing #4chan',
  'pic comparison #boston suspect sunil tripathi fbi released images video missing poster decide',
  'completely convinced sunil tripathi fellow',
  '4chan bombing throwing',
  '4chan thinks pictures bomber'],
 ['boston_fake_03.jpg',
  'boston_fake_23.jpg',
  'boston_fake_34.png',
  'boston_fake_15.jpg',
  'boston_fake_08.jpg'],
 [0, 0, 0, 0, 0])

In [ ]:
train_df=pd.DataFrame(list(zip(train_image_name,train_text,train_target)),columns=['image_name','text','label'])
test_df=pd.DataFrame(list(zip(test_image_name,test_text,test_target)),columns=['image_name','text','label'])
test_df.head()

,image_name,text,label
0,attacks_paris_1.jpg,sadness #bataclan sold musicians & public in o...,0
1,attacks_paris_1.jpg,rt \ @ project40 # last hour spectacular photo...,0
2,attacks_paris_1.jpg,rt \ @ javivi1976 #bataclan tonight before the...,0
3,attacks_paris_1.jpg,rt \ @pizzigatas man has to establish an end f...,0
4,attacks_paris_1.jpg,No Text,0


In [ ]:
print(train_df['label'].value_counts())
print(test_df['label'].value_counts())

0    5839
1    4901
Name: label, dtype: int64
0    518
1    414
Name: label, dtype: int64


# Text Preprocessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def clean_text(text):
    text = re.sub(r"<U\+2019>|<U\+2018>", "\'", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " had ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"n\'t", " not", text)
    text= re.sub(r"<U\+.+>",' ',text)
    text = re.sub(r"[\!\?\.\,\+\-\$\%\^\>\<\=\:\;\*\(\)\{\}\[\]\/\~\&\|\@]", " ", text)
    return text


In [ ]:
words=[]
def create_corpus(messages):
  corpus=[]
  for i in range(len(messages)):
    review=clean_text(messages[i])
    review.lower()
    review=review.split()
    review=[word for word in review if word.isalpha()]
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    for word in review:
      if word not in words:
        words.append(word)
    review=' '.join(review).strip()
    if len(review)==0:
      review='no text'
    corpus.append(review)
  return corpus 


In [ ]:
len(train_text)

10740

In [ ]:
corpus_train=create_corpus(train_text)
corpus_test=create_corpus(test_text)
len(words)

6176

In [ ]:
len(corpus_train),len(corpus_test)

(10740, 932)

In [ ]:
corpus_train[:5]

['need fed solv',
 'pic comparison suspect sunil tripathi fbi releas imag video miss poster decid',
 'complet convinc sunil tripathi fellow',
 'bomb throw',
 'think pictur bomber']

##One Hot Encoding

In [ ]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [ ]:
voc_size=6000
onehot_repr_train=[one_hot(words,voc_size) for words in corpus_train]
onehot_repr_test=[one_hot(words,voc_size) for words in corpus_test]
len(onehot_repr_train),len(onehot_repr_test)

(10740, 932)

In [ ]:
onehot_repr_test

[[1925, 3759, 2515, 997, 2724, 1348, 1146],
 [4339, 3405, 2982, 374, 447, 2463, 1852, 1482, 4976, 3316],
 [4339, 5113, 1852, 5704],
 [4339, 5249, 1161, 2839, 1863, 2363, 5109, 2839, 1863, 2827],
 [3593, 3364],
 [4339, 3854, 1852, 3316, 1482],
 [2648, 1852, 3316],
 [4339, 1644, 2648, 1852, 3316],
 [4339,
  1067,
  2648,
  2463,
  2413,
  3387,
  2662,
  1441,
  4013,
  2261,
  5083,
  4976,
  2272],
 [1924, 3387, 4338, 3854, 4068, 4338, 5159, 1367, 347],
 [5083, 1067, 1852, 3387, 2662, 3054, 4125, 2434],
 [4339, 2806, 2600, 986, 3316, 3387, 1348, 3854, 1852],
 [4339, 1808, 3444, 2285, 986, 3387, 417],
 [4339, 3743, 3387, 4338, 3854, 4068, 4338, 5159, 1367, 347, 2990],
 [1161, 2839, 1863, 2363, 5109, 2839, 1863, 2827],
 [4339, 5249, 1161, 2839, 1863, 2363, 5109, 2839, 1863, 2827],
 [5460],
 [4339, 4621, 3316, 4976],
 [4339, 511, 5603, 5240, 3316, 3387, 1852, 2577, 2776, 4899],
 [4339, 511, 5603, 5240, 3316, 3387, 1852, 2577, 2776, 4899],
 [4339, 4058, 299, 127],
 [2577, 2776, 4899, 4092]

In [ ]:
onehot_repr_train[0],corpus_train[0]

([5633, 1889, 4161], 'need fed solv')

In [ ]:
len(onehot_repr_train+onehot_repr_test)

11672

In [ ]:
max_sent_len=0
for sent in onehot_repr_train+onehot_repr_test:
  l=len(sent)
  if max_sent_len<l:
    max_sent_len=l
max_sent_len

17

In [ ]:
embedded_docs_train=pad_sequences(onehot_repr_train,padding='pre',maxlen=max_sent_len)
embedded_docs_test=pad_sequences(onehot_repr_test,padding='pre',maxlen=max_sent_len)
embedded_docs_train

array([[   0,    0,    0, ..., 5633, 1889, 4161],
       [   0,    0,    0, ...,  508, 4167, 4821],
       [   0,    0,    0, ..., 5052, 1459, 1008],
       ...,
       [   0,    0,    0, ..., 5934, 2302, 1852],
       [   0,    0,    0, ...,   62,  821, 5563],
       [   0,    0,    0, ..., 5240, 4733, 2302]], dtype=int32)

In [ ]:
embedded_docs_train.shape,embedded_docs_test.shape,embedded_docs_train[0]

((10740, 17),
 (932, 17),
 array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 5633, 1889, 4161], dtype=int32))

#Feature Extraction

In [ ]:
#Importing Library
import pickle as pkl
import numpy as np
from os import listdir
from tqdm import tqdm
from keras.models import Model,Sequential
from keras.preprocessing.image import img_to_array,load_img
from keras.applications.vgg19 import VGG19,preprocess_input
from keras.preprocessing import image


In [ ]:
def extract_features(path):
  vgg=VGG19()
  vgg=Model(inputs=vgg.inputs,outputs=vgg.layers[-2].output)
  print(vgg.summary())
  features=dict()
  for name in tqdm(listdir(path)):
    filepath=path+'/'+name
    print(filepath)
    try:
      image=load_img(filepath,target_size=(224,224))
      image=img_to_array(image)
      image=image.reshape(1,image.shape[0],image.shape[1],image.shape[2])
      image=preprocess_input(image)
      feature=vgg.predict(image,verbose=0)
      feature.reshape(-1)
      features[name]=feature
    except Exception as e:
      print(e)
  return features


In [ ]:
train_features=extract_features('/content/drive/MyDrive/mediaeval2016/images_train')
test_features=extract_features('/content/drive/MyDrive/mediaeval2016/images_test')

574717952/574710816 [==============================] - 3s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

  0%|          | 0/411 [00:00<?, ?it/s]

/content/drive/MyDrive/mediaeval2016/images_train/sandy_real_65.jpg


  0%|          | 2/411 [00:33<2:37:04, 23.04s/it]

/content/drive/MyDrive/mediaeval2016/images_train/sandy_real_12.jpg
/content/drive/MyDrive/mediaeval2016/images_train/sandy_real_76.jpg


...

 98%|█████████▊| 127/130 [00:26<00:00,  4.51it/s]

/content/drive/MyDrive/mediaeval2016/images_test/half_everything_1.jpg
/content/drive/MyDrive/mediaeval2016/images_test/convoy_explosion_turkey_3.txt


 99%|█████████▉| 129/130 [00:27<00:00,  5.07it/s]

cannot identify image file <_io.BytesIO object at 0x7fef55dc8770>
/content/drive/MyDrive/mediaeval2016/images_test/refugees_6.jpg
/content/drive/MyDrive/mediaeval2016/images_test/half_everything_9.jpg


100%|██████████| 130/130 [00:27<00:00,  4.72it/s]


In [ ]:
train_features

{'boston_fake_01.jpg': array([[0.8656973, 0.       , 0.5546389, ..., 0.       , 3.0338068,
         0.       ]], dtype=float32),
 'boston_fake_02.jpg': array([[0.6194593, 2.833449 , 2.5446293, ..., 0.       , 0.       ,
         0.       ]], dtype=float32),
 'boston_fake_03.jpg': array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'boston_fake_04.jpg': array([[0.        , 0.        , 0.        , ..., 0.        , 0.91337276,
         0.        ]], dtype=float32),
 'boston_fake_05.jpg': array([[0.       , 0.       , 0.       , ..., 0.       , 1.6767821,
         0.       ]], dtype=float32),
 'boston_fake_06.jpg': array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'boston_fake_07.jpg': array([[0.       , 0.       , 1.1355519, ..., 0.       , 0.       ,
         0.       ]], dtype=float32),
 'boston_fake_08.jpg': array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'boston_fake_09.jpg': array([[0.        , 2.1753638 , 0.07751521, ..., 0.        , 0.        ,
         0.        

In [ ]:
train_features['sandy_real_96.jpg'].shape

(1, 4096)

In [ ]:
pkl.dump(train_features, open('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/mediaeval_train_features.pkl', 'wb'))
pkl.dump(test_features, open('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/mediaeval_test_features.pkl', 'wb'))

#Model

In [ ]:
import pickle as pkl
import numpy as np
#Importing Library
import keras
from keras.layers import Dense,Input,Flatten,Dropout
from keras.models import Model,Sequential
from keras.layers import Embedding,Dense,Flatten,Concatenate,Bidirectional,LSTM
from keras.layers.merge import add
from keras.utils import plot_model
from keras.optimizers import Adam
from keras import regularizers

In [ ]:
train_features=pkl.load(open('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/mediaeval_train_features.pkl','rb'))
test_features=pkl.load(open('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/mediaeval_test_features.pkl','rb'))

In [ ]:
train_image_names=train_df['image_name']
test_image_names=test_df['image_name']
train_image_names[:5],len(train_image_names),len(test_image_names)

(0    boston_fake_03.jpg
 1    boston_fake_23.jpg
 2    boston_fake_34.png
 3    boston_fake_15.jpg
 4    boston_fake_08.jpg
 Name: image_name, dtype: object, 10740, 932)

In [ ]:
train_features_lt=[]
test_features_lt=[]
for train_img in train_image_names:
  feature=train_features[train_img]
  train_features_lt.append(feature.reshape(-1))
for test_img in test_image_names:
  feature=test_features[test_img]
  test_features_lt.append(feature.reshape(-1))
train_features=np.array(train_features_lt)
test_features=np.array(test_features_lt)
train_features.shape,test_features.shape

((10740, 4096), (932, 4096))

In [ ]:
train_features.shape,embedded_docs_train.shape,test_features.shape,embedded_docs_test.shape

((10740, 4096), (10740, 17), (932, 4096), (932, 17))

In [ ]:
max_sent_len=17
voc_size=6000

inputs1=Input(shape=(4096,))
fc_img_1=Dense(256,name='fc_img_1',activation='tanh')(inputs1)
drop_img_1=Dropout(0.4)(fc_img_1)
fc_img_2=Dense(28,name='fc_img_2',activation='tanh')(drop_img_1)


inputs2=Input(shape=(max_sent_len))
txt_embed = Embedding(voc_size, 32, input_length=max_sent_len, name='txt_embed')(inputs2)
lstm_txt_1 = Bidirectional(LSTM(32, return_sequences=True, name='lstm_txt_1', activation='tanh'))(txt_embed)
#lstm_txt_2 = Bidirectional(LSTM(32, return_sequences=True, name='lstm_txt_2', activation='tanh'))(lstm_txt_1)
fc_txt_1 = Dense(48, activation='tanh', name='dense_txt')(lstm_txt_1)
fc_txt_2=Dropout(0.5)(fc_txt_1)
fc_txt = Dense(28, activation='tanh', name='fc_txt_2')(fc_txt_2)

#h = Concatenate(axis=-1, name='concat')([fc_txt, fc_img_2])
h=add([fc_txt,fc_img_2])
h1=Flatten()(h)
h2= Dense(32, name='shared', activation='tanh')(h1)
h3=Dropout(0.4)(h2)

fnd=Dense(32,name='fnd',activation='tanh')(h3)
outputs=Dense(1,activation='sigmoid')(fnd)
model=Model(inputs=[inputs1,inputs2],outputs=outputs)
model.summary()

Model: "model_34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_76 (InputLayer)           [(None, 17)]         0                                            
__________________________________________________________________________________________________
txt_embed (Embedding)           (None, 17, 32)       208000      input_76[0][0]                   
__________________________________________________________________________________________________
bidirectional_57 (Bidirectional (None, 17, 64)       16640       txt_embed[0][0]                  
__________________________________________________________________________________________________
input_75 (InputLayer)           [(None, 4096)]       0                                            
___________________________________________________________________________________________

In [ ]:
#plot_model(model,to_file='model.png')

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
checkpoint_cb=keras.callbacks.ModelCheckpoint("/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/MediaEval_Best_Main_Model1.h5",save_best_only=True,monitor='val_accuracy',mode='auto',verbose=1)

In [ ]:
model.fit(x=[train_features,embedded_docs_train],y=np.array(train_target),
          validation_data=([test_features,embedded_docs_test],np.array(test_target)),
          epochs=60,batch_size=128,callbacks=[checkpoint_cb])

Epoch 1/60
84/84 [==============================] - 6s 20ms/step - loss: 0.1918 - accuracy: 0.9258 - val_loss: 1.1434 - val_accuracy: 0.6609

Epoch 00001: val_accuracy improved from -inf to 0.66094, saving model to /content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/MediaEval_Best_Main_Model1.h5
Epoch 2/60
84/84 [==============================] - 1s 11ms/step - loss: 0.0296 - accuracy: 0.9915 - val_loss: 1.7099 - val_accuracy: 0.5869

Epoch 00002: val_accuracy did not improve from 0.66094
Epoch 3/60
84/84 [==============================] - 1s 11ms/step - loss: 0.0188 - accuracy: 0.9945 - val_loss: 1.3971 - val_accuracy: 0.6438

Epoch 00003: val_accuracy did not improve from 0.66094
Epoch 4/60
84/84 [==============================] - 1s 12ms/step - loss: 0.0148 - accuracy: 0.9957 - val_loss: 1.7767 - val_accuracy: 0.6491

Epoch 00004: val_accuracy did not improve from 0.66094
Epoch 5/60
84/84 [==============================] - 1s 12ms/step - loss: 0.0133 - accuracy: 0.9962 - 

In [ ]:
  #2-->0.80150

In [ ]:
#model.save('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/MediaEval_Best_Main_Model1.h5')

In [ ]:
#del model

In [ ]:
keras.backend.clear_session

<function tensorflow.python.keras.backend.clear_session>

In [ ]:
model=keras.models.load_model('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/MediaEval_Best_Main_Model2.h5')

In [ ]:
#Best_Main_Model5-->0.8947
#4-->0.9278
#3-->0.9388
#2-->0.9418

In [ ]:
test_features=load(open('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/test_features.pkl', 'rb'))

NameError: ignored

In [ ]:
test_lt=list(test_features.values())
test_features=np.array(test_lt)
test_features.shape

(997, 1, 4096)

In [ ]:
test_features.shape,embedded_docs_test.shape

((997, 1, 4096), (997, 37))

In [ ]:
y_pred=model.predict([test_features,embedded_docs_test])
y_pred.shape

(932, 1)

In [ ]:
y_pred[0]

array([0.00369613], dtype=float32)

In [ ]:
y_pred_test=[]
for i in range(len(y_pred)):
  if y_pred[i]<0.5:
    y_pred_test.append(0)
  else:
    y_pred_test.append(1)


In [ ]:
y_pred_test

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
from sklearn.metrics import accuracy_score,classification_report
acc=accuracy_score(np.array(test_target),y_pred_test)
acc

0.8015021459227468

In [ ]:
report=classification_report(test_target,y_pred_test)
print(report)

              precision    recall  f1-score   support

           0       0.83      0.81      0.82       518
           1       0.77      0.79      0.78       414

    accuracy                           0.80       932
   macro avg       0.80      0.80      0.80       932
weighted avg       0.80      0.80      0.80       932

